In [ ]:
# Test near on Postgis

inFeat   = r'D:\indev\OSM2LULC\OSM2LULC_WEB_Via\near_test\all_roads.shp'
nearFeat = r'D:\indev\OSM2LULC\OSM2LULC_WEB_Via\near_test\road_builds.shp'
conParam = {
    'HOST' : 'localhost', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'PORT' : '5432', 'TEMPLATE' : 'postgis_template',
    'DATABASE' : 'lisboa'
}

In [ ]:
import datetime
from gasp.cpu.psql.anls.prox import st_near

time_a = datetime.datetime.now().replace(microsecond=0)

st_near(
    conParam,
    'all_roads',
    'gid', 'geometry',
    'all_builds',
    'geometry', 'teste_near_idx', untilDist=12)

time_b = datetime.datetime.now().replace(microsecond=0)
print time_b - time_a

In [ ]:
import datetime
from gasp.cpu.psql.anls.prox import st_near2

time_a = datetime.datetime.now().replace(microsecond=0)

st_near(conParam, 'all_roads', 'gid', 'geom', 'road_builds', 'geom', 12, 'teste_near1')

time_b = datetime.datetime.now().replace(microsecond=0)
print time_b - time_a

In [ ]:
# Import data from PostGIS to GRASS GIS

from gasp.cpu.grs        import run_grass

grass_base = run_grass(
    r'D:\indev\OSM2LULC\OSM2LULC_WEB_Via\near_test', grassBIN='grass74', location='grloc',
    srs=3857
)

import grass.script as grass
import grass.script.setup as gsetup
gsetup.init(grass_base, r'D:\indev\OSM2LULC\OSM2LULC_WEB_Via\near_test', 'grloc', 'PERMANENT')

In [ ]:
def shp_to_grs(inLyr, outLyr, filterByReg=None, asCMD=None):
    """
    Add Shape to GRASS GIS
    """
    
    if not asCMD:
        from grass.pygrass.modules import Module
        
        f = 'o' if not filterByReg else 'ro'
        
        m = Module(
            "v.in.ogr", input=inLyr, output=outLyr, flags='o',
            overwrite=True, run_=False, quiet=True
        )
        
        m()
    
    else:
        from gasp import exec_cmd
        
        rcmd = exec_cmd((
            "v.in.ogr input={} output={} -o{} --overwrite --quiet"
        ).format(inLyr, outLyr, " -r" if filterByReg else ""))
    
    return outLyr

In [ ]:
from gasp import exec_cmd
        
rcmd = exec_cmd((
    "v.in.ogr input=\"PG:host=localhost dbname=near_tst "
    "user=postgres password=admin port=5432\" "
    "output=teste layer=all_roads -o --overwrite --quiet"
))